In [1]:
from pathlib import Path
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# ["05min", "15min", "30min"]

true_freq = "01min"
sample_freq = "05min"

In [3]:
sub_sample = {"05min": 4, "15min": 12, "30min": 24}
sub_sample_freq = sub_sample[sample_freq]

In [4]:
folder = Path("../Clips_Ines2_results")
experiment = folder / Path("Exp2_01_MedMov_LargeDrop_results")
ground_truth = experiment / Path("01min")
sample = experiment / Path(sample_freq)

In [5]:
dropped_merging = "dropped_merging_trajectories_y0_x0_basic.csv"
dropped_trajectories = "dropped_trajectories_y0_x0_basic.csv"
filtered_trajectories = "filtered_trajectories_y0_x0_basic.csv"
mvt_unfiltered_trajectories = "mvt_unfiltered_trajectories_y0_x0.csv"
unfiltered_trajectories = "unfiltered_trajectories_y0_x0.csv"

In [6]:
true_traj = pd.read_csv(ground_truth / unfiltered_trajectories)
sample_traj = pd.read_csv(sample / unfiltered_trajectories)

# drop all columns that include the word "nr_cells" and "p"
true_traj = true_traj.drop(columns=[col for col in true_traj.columns if "nr_cells" in col or "p" in col]).drop(columns=["full_trajectory_uncertainty"])
sample_traj = sample_traj.drop(columns=[col for col in sample_traj.columns if "nr_cells" in col or "p" in col]).drop(columns=["full_trajectory_uncertainty"])

In [7]:
def swap_columns(df, col1, col2):
    columns = list(df.columns)
    col1_idx, col2_idx = columns.index(col1), columns.index(col2)
    columns[col1_idx], columns[col2_idx] = columns[col2_idx], columns[col1_idx]
    return df[columns]

# Using the function to swap columns 'C' and 'D'
true_traj = swap_columns(true_traj, 'y0', 'x0')
sample_traj = swap_columns(sample_traj, 'y0', 'x0')

In [8]:
one, two = [f'x{i}' for i in range(0, 82, sub_sample_freq)], [f'y{i}' for i in range(0, 82, sub_sample_freq)]
tmp = [f'id_{i}' for i in range(0, 82, sub_sample_freq)] + [item for pair in zip(one, two) for item in pair]
true_traj_sub = true_traj[tmp]

In [9]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

for i in range(0, 82, sub_sample_freq):

    true = true_traj_sub[[f'id_{i}', f'x{i}', f'y{i}']]
    sample = sample_traj[[f'id_{i//sub_sample_freq}', f'x{i//sub_sample_freq}', f'y{i//sub_sample_freq}']]

    common = intersection(true[f'id_{i}'], sample[f'id_{i//sub_sample_freq}'])

    for c in common:
        true_idx = true[true[f'id_{i}'] == c].index[0]
        sample_idx = sample[sample[f'id_{i//sub_sample_freq}'] == c].index[0]

        assert int(true.loc[true_idx, f'x{i}']) == int(sample.loc[sample_idx, f'x{i//sub_sample_freq}'])
        assert int(true.loc[true_idx, f'y{i}']) == int(sample.loc[sample_idx, f'y{i//sub_sample_freq}'])

In [10]:
# iterate over all rows in the sub-sampled data-frame
counter = 0

for ((index_true, traj_true), (index_sample, traj_sample)) in zip(true_traj_sub.iterrows(), sample_traj.iterrows()):

    # count the number of same trajectories by comparing the id's
    same = True
    for i in range(0, 82, sub_sample_freq):
        if int(traj_true[f'id_{i}']) != int(traj_sample[f'id_{i//sub_sample_freq}']):
            same = False
    
    if same:
        counter += 1

print(f'{(counter / len(true_traj_sub) * 100):.5f}')

62.57939


In [11]:
# 05min - 62.57939
# 15min - 53.08751
# 30min - 54.81651